In [8]:
b_dict ={}
a= ['a','b']
b_dict['c']=a 
b_dict
b_dict['d'] = a+a

In [7]:
b_dict['c']

['a', 'b', 'a', 'b']

In [2]:
# Custom module paths

from script.process_file_script import *
from script.process_file_script import read_gvcf_diagnosis_xlsx
from script.functions import *
from script.file_prepare_script import *
from script.Integrated_tools import *

Adding extra dir to sys.path: /Users/liyaqi/PycharmProjects
['/Users/liyaqi/PycharmProjects/PhenoAptAnalysis', '/Users/liyaqi/.pyenv/versions/3.10.7/lib/python310.zip', '/Users/liyaqi/.pyenv/versions/3.10.7/lib/python3.10', '/Users/liyaqi/.pyenv/versions/3.10.7/lib/python3.10/lib-dynload', '', '/Users/liyaqi/.pyenv/versions/3.10.7/envs/jupyterlab/lib/python3.10/site-packages', '/Users/liyaqi/.pyenv/versions/3.10.7/envs/jupyterlab/lib/python3.10/site-packages/aeosa', '/Users/liyaqi/PycharmProjects']


In [7]:
pd.read_csv(f'{pwd}/{filename}/revel_{thresh}/{case_id}.txt', sep="\t")

NameError: name 'thresh' is not defined

In [6]:
filename = 'scoliosis_gVCF_from_zs_updating'
pwd = '/Users/liyaqi/PycharmProjects/PhenoAptAnalysis'
df = read_gvcf_diagnosis_xlsx('/Users/liyaqi/Documents/生信/gVCF-2022-确诊.xlsx')
    
def statistic_to_table(df,pwd,filename,hpo,tools,intersect,REVEL_thresh,CADD_thresh):
    case_ids=df['Blood ID']
    columns_fill = output_columns(hpo,tools,intersect,REVEL_thresh,CADD_thresh)
    part_table = pd.DataFrame(columns=columns_fill)
    part_rr_table = pd.DataFrame(columns=columns_fill+['Frequency','Variant_class','Inheritance_ADAR',f'{hpo}_organ_system',f'{hpo}_organ_system_number'])
    case_id_exomiser_tsv_file_dict = get_case_id_integ_file_map(pwd, filename, 'Exomiser', case_ids, hpo)
    case_id_LIRICAL_tsv_file_dict = get_case_id_integ_file_map(pwd, filename, 'LIRICAL', case_ids, hpo)
    part_table_for_R = pd.DataFrame(columns=columns_fill)
    part_rr_table_for_R = pd.DataFrame(columns=columns_fill+['Frequency','Variant_class','Inheritance_ADAR',f'{hpo}_organ_system',f'{hpo}_organ_system_number'])
    ##for R是后面去掉NA的。

    for i in tqdm(range(len(df))):

        if df.loc[i, hpo][0] == '[':
            hpo_id = df.loc[i, hpo][2:-2]
            hpo_id_input = [k for k in hpo_id.split("', '")]
        else:
            hpo_id = df.loc[i, hpo]
            hpo_id_input = [k for k in hpo_id.split(";")]
        symbol = df.loc[i, "Symbol"]
        entrezGeneId = df.loc[i, 'entrezGeneId']
        ## NCBIGene:6911 entrezGeneId的格式
        entrezId = df.loc[i, 'entrezId']
        ensemblid = df.loc[i, 'Ensembl Gene ID']
        print(ensemblid)
        case_id = df.loc[i, 'Blood ID']
        inheritance = df.loc[i,'Inheritance_ADAR']
        HGNCid=str(int(df.loc[i,'HGNC']))

        filedir,tsv_source_mode = dir_of_Filtered_TSV_MUltiple_source(pwd,filename,case_id)
        if tsv_source_mode=='no Filtered TSV anywhere':
            print(f'{case_id} no tsv file')
            continue

        tsv_candidate_gene_SYMBOL_list = pd.read_csv(f'{pwd}/{filename}/candidategene/{case_id}.txt', sep="\t")
        tsv_candidate_gene_HGNCid_list = pd.read_csv(f'{pwd}/{filename}/candidategene_hgncid/{case_id}.txt', sep="\t")
        tsv_candidate_gene_ensemblid_list = pd.read_csv(f'{pwd}/{filename}/candidategene_ensemblid/{case_id}.txt', sep="\t")

        if len(REVEL_thresh)!=0:
            tsv_candidate_gene_SYMBOL_revel_dict =  {}
            tsv_candidate_gene_HGNCid_revel_dict = {}
            tsv_candidate_gene_ensemblid_revel_dict = {}
            for thresh in REVEL_thresh:
                tsv_candidate_gene_SYMBOL_revel_dict[thresh] =  pd.read_csv(f'{pwd}/{filename}/revel_{thresh}/{case_id}.txt', sep="\t")
                tsv_candidate_gene_HGNCid_revel_dict[thresh] = pd.read_csv(f'{pwd}/{filename}/revel_{thresh}_hgncid/{case_id}.txt', sep="\t")
                tsv_candidate_gene_ensemblid_revel_dict[thresh] = pd.read_csv(f'{pwd}/{filename}/revel_{thresh}_ensemblid/{case_id}.txt', sep="\t")

        if len(CADD_thresh)!=0:
            tsv_candidate_gene_SYMBOL_cadd_dict ={}
            tsv_candidate_gene_HGNCid_cadd_dict = {}
            tsv_candidate_gene_ensemblid_cadd_dict = {}
            for thresh in CADD_thresh:
                tsv_candidate_gene_SYMBOL_cadd_dict[thresh] = pd.read_csv(f'{pwd}/{filename}/cadd_{thresh}/{case_id}.txt', sep="\t")[0]
                tsv_candidate_gene_HGNCid_cadd_dict[thresh] = pd.read_csv(f'{pwd}/{filename}/cadd_{thresh}_hgncid/{case_id}.txt', sep="\t")[0]
                tsv_candidate_gene_ensemblid_cadd_dict[thresh] = pd.read_csv(f'{pwd}/{filename}/cadd_{thresh}_ensemblid/{case_id}.txt', sep="\t")[0]

        part_table.loc[i, 'CaseID'] = case_id
        part_table.loc[i, 'hpo'] = hpo
        part_table.loc[i, 'hpo_id_input'] = f'{hpo_id_input}'
        part_table.loc[i, 'Symbol'] = symbol
        part_rr_table.loc[i, 'CaseID'] = case_id
        part_rr_table.loc[i, 'hpo'] = hpo
        part_rr_table.loc[i, 'hpo_id_input'] = f'{hpo_id_input}'
        part_rr_table.loc[i, 'Symbol'] = symbol
        part_rr_table.loc[i, 'Frequency'] = df.loc[i,'Frequency']
        part_rr_table.loc[i, 'Variant_class'] = df.loc[i, 'Variant_class']
        part_rr_table.loc[i, 'Inheritance_ADAR'] = df.loc[i, 'Inheritance_ADAR']
        part_rr_table.loc[i, f'{hpo}_organ_system'] =df.loc[i,f'{hpo}_organ_system']
        #part_rr_table.loc[i, f'{hpo}_organ_system_number'] = df.loc[i, f'{hpo}_organ_system_number']

        if 'PhenoGenius' in tools:
            PhenoGenius_rank_tsv = pd.read_csv(f'{pwd}/{filename}/{hpo}_Phenogenius_output', sep='\t')
            PhenoGenius_gene_SYMBOL_list = list(PhenoGenius_rank_tsv['gene_symbol'])
            part_table.loc[i,'PhenoGenius_rank'],part_rr_table.loc[i,'PhenoGenius_rank'] = get_rank(symbol,PhenoGenius_gene_SYMBOL_list)
            if intersect:
                PhenoGenius_gene_SYMBOL_intersect_list = PhenoGenius_gene_SYMBOL_list[PhenoGenius_gene_SYMBOL_list.isin(tsv_candidate_gene_SYMBOL_list)]
                part_table.loc[i, 'PhenoGenius_intersect_rank'], part_rr_table.loc[i, 'PhenoGenius_intersect_rank'] = get_rank(symbol,PhenoGenius_gene_SYMBOL_intersect_list)
            if len(REVEL_thresh)!=0:
                for thresh in REVEL_thresh:
                    PhenoGenius_gene_SYMBOL_revel_list = PhenoGenius_gene_SYMBOL_list[PhenoGenius_gene_SYMBOL_list.isin(tsv_candidate_gene_SYMBOL_revel_dict[thresh])]
                    part_table.loc[i, f'PhenoGenius_rank_REVEL_{thresh}'], part_rr_table.loc[i, f'PhenoGenius_rank_REVEL_{thresh}'] = get_rank(symbol,PhenoGenius_gene_SYMBOL_revel_list)
            if len(CADD_thresh)!=0:
                for thresh in CADD_thresh:
                    PhenoGenius_gene_SYMBOL_cadd_list = PhenoGenius_gene_SYMBOL_list[
                        PhenoGenius_gene_SYMBOL_list.isin(tsv_candidate_gene_SYMBOL_cadd_dict[thresh])]
                    part_table.loc[i, f'PhenoGenius_rank_CADD_{thresh}'], part_rr_table.loc[i, f'PhenoGenius_rank_CADD_{thresh}'] = get_rank(symbol,
                                                                                                               PhenoGenius_gene_SYMBOL_cadd_list)
    return part_table,part_rr_table


  0%|                                                    | 0/50 [00:00<?, ?it/s]


ENSG00000149922


TypeError: read_csv() got an unexpected keyword argument 'index'

In [ ]:

a,b=statistic_to_table(df,pwd,filename,'hpo_id',tools=['PhenoGenius'],intersect=True,REVEL_thresh=[0.25,0.5,0.75],CADD_thresh=[10,15,20])


In [5]:
pip install phenoapt suds scikit-allel obonet xlrd==1.2.0 openpyxl


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: /Users/liyaqi/.pyenv/versions/3.10.7/envs/jupyterlab/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
xlrd appscript pyyaml phenoapt

In [8]:
a,b='1'

ValueError: not enough values to unpack (expected 2, got 1)

In [9]:
result_mode_dict = {'PhenoGenius':'Symbol','phrank':'Ensembl Gene ID','phenolyzer':'Symbol','GADO':'Ensembl Gene ID','phen2gene':'Symbol'}


In [12]:
dict_test={}
dict_test['result'] = result_mode_dict

In [14]:
dict_test['result']['PhenoGenius']

'Symbol'

In [28]:
hpo = '233'
case_id = '234'
mode_dict = {'result': {'PhenoGenius': 'Symbol', 'phrank': 'Ensembl Gene ID', 'phenolyzer': 'Symbol',
                                'GADO': 'Ensembl Gene ID'},
                     'dir': {'PhenoGenius': f'{hpo}_PhenoGenius_output/{case_id}.tsv',
                             'phrank': f'phrankoutput/{case_id}_{hpo}_nogenelist_phrank_rank.tsv',
                             'phenolyzer': f'phenolyzeroutput/{case_id}.final_gene_list',
                             'GADO': f'GADOoutput/{case_id}.txt'},
                     'column': {'PhenoGenius': 'gene_symbol', 'phrank': '1', 'phenolyzer': 'ID',
                                'GADO': 'Ensg'}
            }
if 'PhenoGenius' in mode_dict['result'].keys():
    print('ok')

ok


In [2]:
hpo = '233'
case_id = '234'
inheritance = 'AD'
file_dir_dict = {
            'Exomiser':{'original':f'{hpo}_Exomixeroutput/{case_id}_{inheritance}.gene.tsv',
                        'column':'ENTREZ_GENE_ID',
                        'result':'entrezId'},
            'phen2gene':{'original':f'phen2geneoutput_nolist/{case_id}-phen2gene-nolist-output.txt',
                         'column':'ID',
                         'result':'Symbol'},
            'Exomiser1310': {'original': f'{hpo}_Exomixer1310output/{case_id}_{inheritance}.gene.tsv',
                             'column':'ID',
                             'result':'Symbol_inheritance_ADAR'},
            'LIRICAL': {'original': f'{hpo}_LIRICALoutput/{case_id}_{inheritance}.gene.tsv',
                        'column':'entrezGeneId',
                        'result':'entrezGeneId'}
        }

file_dir_dict.keys()

dict_keys(['Exomiser', 'phen2gene', 'Exomiser1310', 'LIRICAL'])

In [18]:
file_dir_dict[list(file_dir_dict.keys())[0]].keys()

dict_keys(['original', 'column', 'result'])

In [20]:
import logging
logger = logging.getLogger(__name__)
logger.info('like you')

In [22]:
print(logger)

<Logger __main__ (WARNING)>


In [15]:
with open('/Users/liyaqi/test.txt') as f:
    list = f.read().split(',')
    print(list)

['WASH7P', 'WASH7P', 'DDX11L1', 'DDX11L1', 'WASH7P', 'DDX11L1', 'DDX11L1', 'WASH7P', 'WASH7P', 'SDF4', 'SDF4', 'B3GALT6', 'SDF4', 'SDF4', 'SDF4', 'SDF4', 'SDF4', 'SDF4', 'B3GALT6', 'SDF4', 'SDF4', 'SDF4', 'SDF4', 'CPSF3L', 'ACAP3', 'ACAP3', 'ACAP3', 'PUSL1', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'ACAP3', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'PUSL1', 'CPSF3L', 'PUSL1', 'CPSF3L', 'PUSL1', 'ACAP3', 'CPSF3L', 'ACAP3', 'CPSF3L', 'CPSF3L', 'ACAP3', 'PUSL1', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'GLTPD1', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'RP5-890O3.9', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'GLTPD1', 'CPSF3L', 'GLTPD1', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPSF3L', 'CPS

In [12]:
str(r)

<_io.TextIOWrapper name='/Users/liyaqi/test.txt' mode='r' encoding='UTF-8'>
